# 🚀 Tutorial Completo: Spark + Airflow + MLflow

Este notebook te guiará paso a paso en el uso del pipeline de ML usando Spark para procesamiento, Airflow para orquestación y MLflow para tracking.

## 📋 Contenido
1. [Setup Inicial](#setup)
2. [Introducción a Spark](#spark-intro)
3. [Procesamiento de Datos con Spark](#spark-processing)
4. [Entrenamiento de Modelos con Spark MLlib](#spark-ml)
5. [Tracking con MLflow](#mlflow)
6. [Orquestación con Airflow](#airflow)
7. [Pipeline Completo End-to-End](#pipeline)
8. [Monitoreo y Debugging](#monitoring)

---

## 1. Setup Inicial <a id="setup"></a>

### Verificar que los servicios estén corriendo

In [ ]:
import requests
import os
from datetime import datetime

# Verificar servicios
services = {
    "Airflow": "http://localhost:8080/health",
    "MLflow": "http://localhost:5000/health",
    "Spark Master": "http://localhost:8081"
}

print("🔍 Verificando servicios...\n")
for name, url in services.items():
    try:
        response = requests.get(url, timeout=5)
        status = "✅ OK" if response.status_code == 200 else f"⚠️ {response.status_code}"
        print(f"{name}: {status}")
    except Exception as e:
        print(f"{name}: ❌ No disponible - {str(e)[:50]}")

print("\n" + "="*60)

### Instalar dependencias necesarias

In [ ]:
# Instalar paquetes si no están instalados
!pip install -q pyspark mlflow pandas numpy matplotlib seaborn scikit-learn

### Importar librerías

In [ ]:
# Imports principales
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

import mlflow
import mlflow.spark
from mlflow.tracking import MlflowClient

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from requests.auth import HTTPBasicAuth

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Librerías importadas correctamente")

---
## 2. Introducción a Spark <a id="spark-intro"></a>

### ¿Qué es Apache Spark?

Apache Spark es un motor de procesamiento distribuido diseñado para:
- **Procesamiento en memoria**: 100x más rápido que MapReduce
- **Escalabilidad**: De laptop a cluster de miles de nodos
- **APIs unificadas**: Batch, streaming, ML, grafos
- **Multi-lenguaje**: Scala, Java, Python, R

### Arquitectura de Spark

```
┌─────────────────────────────────────────┐
│          Driver Program                 │
│     (SparkContext/SparkSession)         │
└────────────┬────────────────────────────┘
             │
             ▼
┌─────────────────────────────────────────┐
│        Cluster Manager                  │
│  (Standalone / YARN / Kubernetes)       │
└────┬───────────┬────────────┬───────────┘
     │           │            │
     ▼           ▼            ▼
┌─────────┐ ┌─────────┐ ┌─────────┐
│Worker   │ │Worker   │ │Worker   │
│Executor │ │Executor │ │Executor │
└─────────┘ └─────────┘ └─────────┘
```

### Crear Sesión de Spark

In [ ]:
# Crear SparkSession
spark = SparkSession.builder \
    .appName("Tutorial-Spark-Airflow") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("✅ Spark Session creada")
print(f"   Spark Version: {spark.version}")
print(f"   Master: {spark.sparkContext.master}")
print(f"   App Name: {spark.sparkContext.appName}")
print(f"\n   Spark UI: http://localhost:4040")

---
## 3. Procesamiento de Datos con Spark <a id="spark-processing"></a>

### 3.1 Crear Dataset de Ejemplo

Vamos a crear un dataset de ventas simulado para demostrar las capacidades de Spark.

In [ ]:
# Generar datos de ejemplo con pandas
np.random.seed(42)

n_records = 10000

data = {
    'fecha': pd.date_range('2023-01-01', periods=n_records, freq='H'),
    'producto_id': np.random.randint(1, 101, n_records),
    'categoria': np.random.choice(['Electrónica', 'Ropa', 'Alimentos', 'Hogar'], n_records),
    'cantidad': np.random.randint(1, 50, n_records),
    'precio_unitario': np.random.uniform(10, 1000, n_records).round(2),
    'descuento': np.random.choice([0, 5, 10, 15, 20], n_records),
    'cliente_id': np.random.randint(1, 1001, n_records),
    'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], n_records)
}

df_pandas = pd.DataFrame(data)

# Calcular columnas derivadas
df_pandas['precio_con_descuento'] = df_pandas['precio_unitario'] * (1 - df_pandas['descuento']/100)
df_pandas['total'] = df_pandas['cantidad'] * df_pandas['precio_con_descuento']
df_pandas['mes'] = df_pandas['fecha'].dt.month
df_pandas['dia_semana'] = df_pandas['fecha'].dt.dayofweek
df_pandas['hora'] = df_pandas['fecha'].dt.hour

print(f"✅ Dataset creado: {len(df_pandas):,} registros")
print(f"   Rango de fechas: {df_pandas['fecha'].min()} a {df_pandas['fecha'].max()}")
print(f"   Total de ventas: ${df_pandas['total'].sum():,.2f}")

df_pandas.head()

### 3.2 Convertir a Spark DataFrame

In [ ]:
# Convertir pandas DataFrame a Spark DataFrame
df_spark = spark.createDataFrame(df_pandas)

print("✅ DataFrame de Spark creado")
print(f"   Particiones: {df_spark.rdd.getNumPartitions()}")
print(f"   Registros: {df_spark.count():,}")

# Mostrar esquema
print("\n📋 Esquema del DataFrame:")
df_spark.printSchema()

In [ ]:
# Mostrar primeras filas
print("📊 Primeras 5 filas:")
df_spark.show(5, truncate=False)

### 3.3 Operaciones Básicas con Spark

#### Filtrado

In [ ]:
# Filtrar ventas con descuento > 10%
ventas_descuento = df_spark.filter(df_spark.descuento > 10)

print(f"📌 Ventas con descuento > 10%: {ventas_descuento.count():,}")
print(f"   Porcentaje: {ventas_descuento.count() / df_spark.count() * 100:.1f}%")

#### Agregaciones

In [ ]:
from pyspark.sql import functions as F

# Ventas por categoría
ventas_por_categoria = df_spark.groupBy('categoria').agg(
    F.count('*').alias('num_ventas'),
    F.sum('total').alias('total_ventas'),
    F.avg('total').alias('promedio_venta'),
    F.min('total').alias('min_venta'),
    F.max('total').alias('max_venta')
).orderBy(F.desc('total_ventas'))

print("📊 Ventas por Categoría:")
ventas_por_categoria.show(truncate=False)

#### Joins

In [ ]:
# Crear tabla de productos
productos_data = [
    (1, 'Laptop', 'Electrónica', 'Premium'),
    (2, 'Mouse', 'Electrónica', 'Básico'),
    (3, 'Camisa', 'Ropa', 'Básico'),
    (4, 'Arroz', 'Alimentos', 'Básico'),
    (5, 'Refrigerador', 'Hogar', 'Premium')
]

productos_schema = ['producto_id', 'nombre', 'categoria', 'segmento']
df_productos = spark.createDataFrame(productos_data, productos_schema)

# Join con tabla de productos (solo para demostración con IDs 1-5)
df_con_productos = df_spark.filter(F.col('producto_id') <= 5) \
    .join(df_productos, 'producto_id', 'left')

print("🔗 DataFrame con Join:")
df_con_productos.select('producto_id', 'nombre', 'cantidad', 'total', 'segmento').show(10)

#### Window Functions

In [ ]:
from pyspark.sql.window import Window

# Top 3 ventas por categoría
window_spec = Window.partitionBy('categoria').orderBy(F.desc('total'))

df_top_ventas = df_spark.withColumn('rank', F.row_number().over(window_spec)) \
    .filter(F.col('rank') <= 3) \
    .select('categoria', 'producto_id', 'total', 'rank') \
    .orderBy('categoria', 'rank')

print("🏆 Top 3 Ventas por Categoría:")
df_top_ventas.show(20)

### 3.4 Análisis Temporal

In [ ]:
# Ventas por mes
ventas_mensuales = df_spark.groupBy('mes').agg(
    F.sum('total').alias('total_mes'),
    F.count('*').alias('num_transacciones'),
    F.avg('total').alias('ticket_promedio')
).orderBy('mes')

print("📅 Ventas Mensuales:")
ventas_mensuales.show(12)

# Convertir a pandas para visualización
ventas_mensuales_pd = ventas_mensuales.toPandas()

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].bar(ventas_mensuales_pd['mes'], ventas_mensuales_pd['total_mes'])
axes[0].set_xlabel('Mes')
axes[0].set_ylabel('Total Ventas ($)')
axes[0].set_title('Ventas Totales por Mes')
axes[0].grid(True, alpha=0.3)

axes[1].plot(ventas_mensuales_pd['mes'], ventas_mensuales_pd['ticket_promedio'], marker='o')
axes[1].set_xlabel('Mes')
axes[1].set_ylabel('Ticket Promedio ($)')
axes[1].set_title('Ticket Promedio por Mes')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.5 Guardar Datos Procesados

In [ ]:
# Crear directorio de datos si no existe
!mkdir -p /tmp/spark_data

# Guardar en formato Parquet (optimizado)
output_path = "/tmp/spark_data/ventas_procesadas.parquet"
df_spark.write.mode('overwrite').parquet(output_path)

print(f"✅ Datos guardados en: {output_path}")

# Verificar que se guardó correctamente
df_leido = spark.read.parquet(output_path)
print(f"   Registros leídos: {df_leido.count():,}")

---
## 4. Entrenamiento de Modelos con Spark MLlib <a id="spark-ml"></a>

### 4.1 Preparación de Datos para ML

In [ ]:
# Crear features para predecir el total de venta
# Features: cantidad, precio_unitario, descuento, mes, dia_semana, hora

# Primero, convertir categorías a numéricas
from pyspark.ml.feature import StringIndexer

categoria_indexer = StringIndexer(inputCol='categoria', outputCol='categoria_idx')
region_indexer = StringIndexer(inputCol='region', outputCol='region_idx')

# Aplicar indexers
df_ml = categoria_indexer.fit(df_spark).transform(df_spark)
df_ml = region_indexer.fit(df_ml).transform(df_ml)

# Seleccionar features numéricas
feature_cols = [
    'cantidad', 'precio_unitario', 'descuento', 
    'mes', 'dia_semana', 'hora',
    'categoria_idx', 'region_idx'
]

# Ensamblar features en un vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features_raw')
df_ml = assembler.transform(df_ml)

# Escalar features
scaler = StandardScaler(inputCol='features_raw', outputCol='features', withMean=True, withStd=True)
scaler_model = scaler.fit(df_ml)
df_ml = scaler_model.transform(df_ml)

# Seleccionar solo lo necesario
df_ml = df_ml.select('features', F.col('total').alias('label'))

print("✅ Features preparadas")
print("\n📊 Muestra de datos:")
df_ml.show(5, truncate=False)

### 4.2 Dividir en Train/Test

In [ ]:
# Split 80/20
train_data, test_data = df_ml.randomSplit([0.8, 0.2], seed=42)

print(f"📊 División de datos:")
print(f"   Train: {train_data.count():,} registros")
print(f"   Test:  {test_data.count():,} registros")

# Cache para mejor rendimiento
train_data.cache()
test_data.cache()

print("\n✅ Datos cacheados en memoria")

### 4.3 Entrenar Modelo - Regresión Lineal

In [ ]:
# Crear modelo de regresión lineal
lr = LinearRegression(
    featuresCol='features',
    labelCol='label',
    maxIter=10,
    regParam=0.01,
    elasticNetParam=0.0
)

print("🔄 Entrenando modelo de Regresión Lineal...")
lr_model = lr.fit(train_data)

print("\n✅ Modelo entrenado")
print(f"   Coeficientes: {len(lr_model.coefficients)}")
print(f"   Intercepto: {lr_model.intercept:.2f}")
print(f"   RMSE (train): {lr_model.summary.rootMeanSquaredError:.2f}")
print(f"   R² (train): {lr_model.summary.r2:.4f}")

### 4.4 Evaluar Modelo

In [ ]:
# Predecir en test set
predictions_lr = lr_model.transform(test_data)

# Evaluar
evaluator_rmse = RegressionEvaluator(labelCol='label', predictionCol='prediction', metricName='rmse')
evaluator_r2 = RegressionEvaluator(labelCol='label', predictionCol='prediction', metricName='r2')
evaluator_mae = RegressionEvaluator(labelCol='label', predictionCol='prediction', metricName='mae')

rmse_lr = evaluator_rmse.evaluate(predictions_lr)
r2_lr = evaluator_r2.evaluate(predictions_lr)
mae_lr = evaluator_mae.evaluate(predictions_lr)

print("📊 Métricas del Modelo - Regresión Lineal:")
print(f"   RMSE: {rmse_lr:,.2f}")
print(f"   MAE:  {mae_lr:,.2f}")
print(f"   R²:   {r2_lr:.4f}")

# Mostrar algunas predicciones
print("\n🔮 Predicciones vs Real:")
predictions_lr.select('label', 'prediction').show(10)

### 4.5 Entrenar Modelo - Random Forest

In [ ]:
# Crear modelo Random Forest
rf = RandomForestRegressor(
    featuresCol='features',
    labelCol='label',
    numTrees=20,
    maxDepth=5,
    seed=42
)

print("🔄 Entrenando modelo Random Forest...")
rf_model = rf.fit(train_data)

print("\n✅ Random Forest entrenado")
print(f"   Número de árboles: {rf_model.getNumTrees}")
print(f"   Profundidad máxima: {rf_model.getMaxDepth()}")

# Evaluar
predictions_rf = rf_model.transform(test_data)

rmse_rf = evaluator_rmse.evaluate(predictions_rf)
r2_rf = evaluator_r2.evaluate(predictions_rf)
mae_rf = evaluator_mae.evaluate(predictions_rf)

print("\n📊 Métricas del Modelo - Random Forest:")
print(f"   RMSE: {rmse_rf:,.2f}")
print(f"   MAE:  {mae_rf:,.2f}")
print(f"   R²:   {r2_rf:.4f}")

### 4.6 Comparar Modelos

In [ ]:
# Crear tabla comparativa
comparison_df = pd.DataFrame({
    'Modelo': ['Regresión Lineal', 'Random Forest'],
    'RMSE': [rmse_lr, rmse_rf],
    'MAE': [mae_lr, mae_rf],
    'R²': [r2_lr, r2_rf]
})

print("📊 Comparación de Modelos:")
print(comparison_df.to_string(index=False))

# Visualizar comparación
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

metrics = ['RMSE', 'MAE', 'R²']
for i, metric in enumerate(metrics):
    axes[i].bar(comparison_df['Modelo'], comparison_df[metric], color=['#3498db', '#e74c3c'])
    axes[i].set_title(f'{metric} por Modelo')
    axes[i].set_ylabel(metric)
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Determinar mejor modelo
mejor_modelo = 'Random Forest' if r2_rf > r2_lr else 'Regresión Lineal'
print(f"\n🏆 Mejor modelo: {mejor_modelo}")

---
## 5. Tracking con MLflow <a id="mlflow"></a>

### 5.1 Configurar MLflow

In [ ]:
# Configurar MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("spark-airflow-tutorial")

print("✅ MLflow configurado")
print(f"   Tracking URI: http://localhost:5000")
print(f"   Experimento: spark-airflow-tutorial")

### 5.2 Registrar Experimento - Regresión Lineal

In [ ]:
# Registrar experimento con MLflow
with mlflow.start_run(run_name="LinearRegression-Tutorial") as run:
    # Log de parámetros
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("maxIter", 10)
    mlflow.log_param("regParam", 0.01)
    mlflow.log_param("train_size", train_data.count())
    mlflow.log_param("test_size", test_data.count())
    mlflow.log_param("features", feature_cols)
    
    # Log de métricas
    mlflow.log_metric("rmse", rmse_lr)
    mlflow.log_metric("mae", mae_lr)
    mlflow.log_metric("r2", r2_lr)
    
    # Log del modelo
    mlflow.spark.log_model(lr_model, "model")
    
    run_id_lr = run.info.run_id
    
    print(f"✅ Experimento registrado en MLflow")
    print(f"   Run ID: {run_id_lr}")
    print(f"   Ver en: http://localhost:5000/#/experiments/1/runs/{run_id_lr}")

### 5.3 Registrar Experimento - Random Forest

In [ ]:
# Registrar Random Forest
with mlflow.start_run(run_name="RandomForest-Tutorial") as run:
    # Log de parámetros
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("numTrees", 20)
    mlflow.log_param("maxDepth", 5)
    mlflow.log_param("train_size", train_data.count())
    mlflow.log_param("test_size", test_data.count())
    mlflow.log_param("features", feature_cols)
    
    # Log de métricas
    mlflow.log_metric("rmse", rmse_rf)
    mlflow.log_metric("mae", mae_rf)
    mlflow.log_metric("r2", r2_rf)
    
    # Feature importance
    importance_dict = {f"feature_{i}": float(imp) for i, imp in enumerate(rf_model.featureImportances)}
    mlflow.log_dict(importance_dict, "feature_importance.json")
    
    # Log del modelo
    mlflow.spark.log_model(rf_model, "model")
    
    run_id_rf = run.info.run_id
    
    print(f"✅ Experimento Random Forest registrado")
    print(f"   Run ID: {run_id_rf}")
    print(f"   Ver en: http://localhost:5000/#/experiments/1/runs/{run_id_rf}")

### 5.4 Comparar Experimentos en MLflow

In [ ]:
# Obtener todos los runs del experimento
client = MlflowClient()
experiment = client.get_experiment_by_name("spark-airflow-tutorial")

if experiment:
    runs = client.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["metrics.r2 DESC"]
    )
    
    print("📊 Experimentos en MLflow:\n")
    for run in runs:
        print(f"Run: {run.info.run_name}")
        print(f"  Run ID: {run.info.run_id}")
        print(f"  RMSE: {run.data.metrics.get('rmse', 'N/A')}")
        print(f"  R²: {run.data.metrics.get('r2', 'N/A')}")
        print(f"  Modelo: {run.data.params.get('model_type', 'N/A')}")
        print()
else:
    print("⚠️ Experimento no encontrado")

---
## 6. Orquestación con Airflow <a id="airflow"></a>

### 6.1 Listar DAGs Disponibles

In [ ]:
# Configuración de Airflow API
AIRFLOW_API_URL = "http://localhost:8080/api/v1"
AIRFLOW_AUTH = HTTPBasicAuth("airflow", "airflow")

# Listar DAGs
try:
    response = requests.get(f"{AIRFLOW_API_URL}/dags", auth=AIRFLOW_AUTH)
    if response.status_code == 200:
        dags = response.json()['dags']
        print(f"📋 DAGs disponibles ({len(dags)}):")
        print()
        for dag in dags:
            status = "✅ Activo" if dag['is_active'] and not dag['is_paused'] else "⏸️ Pausado"
            print(f"  {status} - {dag['dag_id']}")
            print(f"    Schedule: {dag.get('schedule_interval', 'None')}")
            print(f"    Tags: {', '.join(dag.get('tags', []))}")
            print()
    else:
        print(f"❌ Error: {response.status_code}")
except Exception as e:
    print(f"❌ Error conectando con Airflow: {e}")

### 6.2 Disparar DAG de ML Pipeline

In [ ]:
# Disparar el DAG ml_pipeline
dag_id = "ml_pipeline"

try:
    # Trigger DAG
    trigger_url = f"{AIRFLOW_API_URL}/dags/{dag_id}/dagRuns"
    payload = {
        "conf": {
            "triggered_from": "jupyter_notebook",
            "timestamp": datetime.now().isoformat()
        }
    }
    
    response = requests.post(
        trigger_url,
        json=payload,
        auth=AIRFLOW_AUTH,
        headers={"Content-Type": "application/json"}
    )
    
    if response.status_code == 200:
        result = response.json()
        dag_run_id = result['dag_run_id']
        print(f"✅ DAG disparado exitosamente")
        print(f"   DAG ID: {dag_id}")
        print(f"   Run ID: {dag_run_id}")
        print(f"   Estado: {result['state']}")
        print(f"\n   Ver en Airflow: http://localhost:8080/dags/{dag_id}/grid")
    else:
        print(f"❌ Error disparando DAG: {response.status_code}")
        print(f"   Respuesta: {response.text}")
except Exception as e:
    print(f"❌ Error: {e}")

### 6.3 Monitorear Estado del DAG

In [ ]:
import time

def get_dag_status(dag_id, max_checks=10):
    """Monitorea el estado de un DAG run"""
    
    # Obtener el último run
    runs_url = f"{AIRFLOW_API_URL}/dags/{dag_id}/dagRuns"
    response = requests.get(runs_url, auth=AIRFLOW_AUTH, params={"limit": 1})
    
    if response.status_code != 200:
        print("❌ No se pudo obtener el estado del DAG")
        return
    
    runs = response.json()['dag_runs']
    if not runs:
        print("⚠️ No hay runs para este DAG")
        return
    
    latest_run = runs[0]
    dag_run_id = latest_run['dag_run_id']
    
    print(f"🔄 Monitoreando DAG Run: {dag_run_id}\n")
    
    for i in range(max_checks):
        # Obtener estado de tareas
        tasks_url = f"{AIRFLOW_API_URL}/dags/{dag_id}/dagRuns/{dag_run_id}/taskInstances"
        response = requests.get(tasks_url, auth=AIRFLOW_AUTH)
        
        if response.status_code == 200:
            tasks = response.json()['task_instances']
            
            print(f"\r📊 Check {i+1}/{max_checks}:", end="")
            
            states = {}
            for task in tasks:
                state = task['state']
                states[state] = states.get(state, 0) + 1
            
            status_str = " | ".join([f"{k}: {v}" for k, v in states.items()])
            print(f" {status_str}", end="")
            
            # Verificar si terminó
            if 'running' not in states and 'queued' not in states:
                print("\n\n✅ DAG completado")
                break
            
            time.sleep(5)
        else:
            print(f"\n❌ Error obteniendo estado: {response.status_code}")
            break
    
    # Mostrar estado final
    print("\n📋 Estado Final de Tareas:\n")
    for task in tasks:
        emoji = {
            'success': '✅',
            'failed': '❌',
            'running': '🔄',
            'queued': '⏳',
            'skipped': '⏭️'
        }.get(task['state'], '❓')
        
        print(f"{emoji} {task['task_id']}: {task['state']}")
        if task.get('duration'):
            print(f"   Duración: {task['duration']:.2f}s")

# Ejecutar monitoreo
get_dag_status("ml_pipeline")

### 6.4 Crear DAG Personalizado Programáticamente

In [ ]:
# Código de ejemplo para crear un DAG personalizado
dag_code = '''
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.operators.bash import BashOperator

default_args = {
    'owner': 'tutorial',
    'depends_on_past': False,
    'start_date': datetime(2024, 1, 1),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'custom_spark_pipeline',
    default_args=default_args,
    description='Pipeline personalizado de Spark',
    schedule_interval='@daily',
    catchup=False,
    tags=['spark', 'tutorial', 'custom'],
)

def load_data(**context):
    """Carga datos desde fuente"""
    print("Cargando datos...")
    # Aquí iría la lógica de carga
    return {"records": 10000, "status": "success"}

def process_with_spark(**context):
    """Procesa datos con Spark"""
    from pyspark.sql import SparkSession
    
    spark = SparkSession.builder.appName("CustomPipeline").getOrCreate()
    print(f"Spark Version: {spark.version}")
    # Procesamiento...
    spark.stop()
    return {"processed": True}

def train_model(**context):
    """Entrena modelo ML"""
    print("Entrenando modelo...")
    # Lógica de entrenamiento
    return {"model_id": "model_v1.0", "accuracy": 0.95}

# Definir tareas
t1 = PythonOperator(
    task_id='load_data',
    python_callable=load_data,
    dag=dag,
)

t2 = PythonOperator(
    task_id='process_spark',
    python_callable=process_with_spark,
    dag=dag,
)

t3 = PythonOperator(
    task_id='train_model',
    python_callable=train_model,
    dag=dag,
)

t4 = BashOperator(
    task_id='notify',
    bash_command='echo "Pipeline completado exitosamente"',
    dag=dag,
)

# Dependencias: load -> process -> train -> notify
t1 >> t2 >> t3 >> t4
'''

print("📝 Código de DAG Personalizado:")
print("\nPara crear este DAG:")
print("1. Copia el código en: airflow/dags/custom_spark_pipeline.py")
print("2. Espera ~30 segundos para que Airflow lo detecte")
print("3. Actualiza la UI de Airflow")
print("\n" + "="*60)
print(dag_code)
print("="*60)

---
## 7. Pipeline Completo End-to-End <a id="pipeline"></a>

### 7.1 Definir Pipeline Completo

In [ ]:
def run_complete_pipeline(data_path=None, model_type='RandomForest'):
    """
    Ejecuta el pipeline completo de ML:
    1. Carga de datos
    2. Procesamiento con Spark
    3. Entrenamiento de modelo
    4. Evaluación
    5. Registro en MLflow
    """
    
    print("🚀 Iniciando Pipeline Completo\n")
    print("="*60)
    
    # Paso 1: Cargar datos
    print("\n[1/5] 📥 Cargando datos...")
    if data_path:
        df = spark.read.parquet(data_path)
        print(f"      ✅ Datos cargados desde {data_path}")
    else:
        df = df_spark  # Usar datos del tutorial
        print(f"      ✅ Usando datos del tutorial")
    
    print(f"      Registros: {df.count():,}")
    
    # Paso 2: Procesamiento
    print("\n[2/5] ⚙️ Procesando datos...")
    
    # Indexar categorías
    categoria_idx = StringIndexer(inputCol='categoria', outputCol='categoria_idx')
    region_idx = StringIndexer(inputCol='region', outputCol='region_idx')
    
    df_processed = categoria_idx.fit(df).transform(df)
    df_processed = region_idx.fit(df_processed).transform(df_processed)
    
    # Ensamblar features
    feature_cols = ['cantidad', 'precio_unitario', 'descuento', 
                   'mes', 'dia_semana', 'hora', 'categoria_idx', 'region_idx']
    assembler = VectorAssembler(inputCols=feature_cols, outputCol='features_raw')
    df_processed = assembler.transform(df_processed)
    
    # Escalar
    scaler = StandardScaler(inputCol='features_raw', outputCol='features', 
                           withMean=True, withStd=True)
    scaler_model = scaler.fit(df_processed)
    df_ml = scaler_model.transform(df_processed)
    df_ml = df_ml.select('features', F.col('total').alias('label'))
    
    print(f"      ✅ Datos procesados")
    
    # Paso 3: Split
    print("\n[3/5] ✂️ Dividiendo datos...")
    train, test = df_ml.randomSplit([0.8, 0.2], seed=42)
    train.cache()
    test.cache()
    print(f"      Train: {train.count():,} | Test: {test.count():,}")
    
    # Paso 4: Entrenar
    print(f"\n[4/5] 🎓 Entrenando modelo {model_type}...")
    
    if model_type == 'LinearRegression':
        model = LinearRegression(maxIter=10, regParam=0.01)
    else:  # RandomForest
        model = RandomForestRegressor(numTrees=20, maxDepth=5, seed=42)
    
    trained_model = model.fit(train)
    print(f"      ✅ Modelo entrenado")
    
    # Paso 5: Evaluar
    print("\n[5/5] 📊 Evaluando modelo...")
    predictions = trained_model.transform(test)
    
    evaluator_rmse = RegressionEvaluator(labelCol='label', predictionCol='prediction', 
                                         metricName='rmse')
    evaluator_r2 = RegressionEvaluator(labelCol='label', predictionCol='prediction', 
                                       metricName='r2')
    evaluator_mae = RegressionEvaluator(labelCol='label', predictionCol='prediction', 
                                        metricName='mae')
    
    rmse = evaluator_rmse.evaluate(predictions)
    r2 = evaluator_r2.evaluate(predictions)
    mae = evaluator_mae.evaluate(predictions)
    
    print(f"      RMSE: {rmse:,.2f}")
    print(f"      R²:   {r2:.4f}")
    print(f"      MAE:  {mae:,.2f}")
    
    # Paso 6: Registrar en MLflow
    print("\n[6/6] 📝 Registrando en MLflow...")
    
    with mlflow.start_run(run_name=f"{model_type}-Pipeline-{datetime.now().strftime('%Y%m%d_%H%M%S')}") as run:
        mlflow.log_param("model_type", model_type)
        mlflow.log_param("train_size", train.count())
        mlflow.log_param("test_size", test.count())
        mlflow.log_param("features", feature_cols)
        
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        
        mlflow.spark.log_model(trained_model, "model")
        
        run_id = run.info.run_id
        print(f"      ✅ Registrado con Run ID: {run_id}")
    
    print("\n" + "="*60)
    print("✅ Pipeline completado exitosamente\n")
    
    return {
        'model': trained_model,
        'metrics': {'rmse': rmse, 'r2': r2, 'mae': mae},
        'run_id': run_id
    }

print("✅ Función de pipeline definida")

### 7.2 Ejecutar Pipeline

In [ ]:
# Ejecutar pipeline completo
result = run_complete_pipeline(model_type='RandomForest')

print("\n📊 Resumen del Pipeline:")
print(f"   Modelo: RandomForest")
print(f"   RMSE: {result['metrics']['rmse']:,.2f}")
print(f"   R²: {result['metrics']['r2']:.4f}")
print(f"   MLflow Run: {result['run_id']}")
print(f"\n   Ver en MLflow: http://localhost:5000/#/experiments/1/runs/{result['run_id']}")

---
## 8. Monitoreo y Debugging <a id="monitoring"></a>

### 8.1 Monitorear Spark UI

In [ ]:
print("🔍 URLs de Monitoreo:\n")
print("Spark:")
print("  • Spark UI (local): http://localhost:4040")
print("  • Spark Master UI: http://localhost:8081")
print()
print("Airflow:")
print("  • Airflow UI: http://localhost:8080")
print("  • Usuario: airflow / Contraseña: airflow")
print()
print("MLflow:")
print("  • MLflow UI: http://localhost:5000")
print()
print("PostgreSQL:")
print("  • Host: localhost:5433")
print("  • Usuario: airflow / Contraseña: airflow")

### 8.2 Verificar Recursos de Spark

In [ ]:
# Información de Spark Context
sc = spark.sparkContext

print("⚡ Información de Spark Context:\n")
print(f"  App Name: {sc.appName}")
print(f"  Master: {sc.master}")
print(f"  Spark Version: {sc.version}")
print(f"  Default Parallelism: {sc.defaultParallelism}")
print(f"  Default Min Partitions: {sc.defaultMinPartitions}")
print()
print(f"  Configuración actual:")
for key, value in sc.getConf().getAll():
    if 'memory' in key.lower() or 'executor' in key.lower():
        print(f"    {key}: {value}")

### 8.3 Estadísticas de Ejecución

In [ ]:
# Obtener estadísticas de los últimos jobs
status_tracker = sc.statusTracker()

print("📊 Estadísticas de Jobs:\n")
active_jobs = status_tracker.getActiveJobIds()
print(f"  Jobs activos: {len(active_jobs)}")

# Pool info
print(f"\n  Pool: {sc.getLocalProperty('spark.scheduler.pool') or 'default'}")

print("\n✅ Para más detalles, visita Spark UI: http://localhost:4040")

### 8.4 Cleanup

In [ ]:
# Limpiar datos cacheados
try:
    train_data.unpersist()
    test_data.unpersist()
    print("✅ Cache limpiado")
except:
    print("⚠️ No hay datos cacheados")

# Opcional: Detener Spark Session
# spark.stop()
# print("✅ Spark Session detenida")

---
## 🎯 Resumen del Tutorial

### Lo que aprendiste:

1. **Spark Fundamentals**:
   - Crear SparkSession
   - DataFrames y operaciones
   - Agregaciones y Joins
   - Window Functions
   - Guardar/cargar datos en Parquet

2. **Spark MLlib**:
   - Feature engineering
   - Entrenamiento de modelos (LR, RF)
   - Evaluación de modelos
   - Comparación de modelos

3. **MLflow**:
   - Tracking de experimentos
   - Log de parámetros y métricas
   - Registro de modelos
   - Comparación de runs

4. **Airflow**:
   - Listar DAGs
   - Disparar DAGs programáticamente
   - Monitorear ejecución
   - Crear DAGs personalizados

5. **Pipeline End-to-End**:
   - Integrar todos los componentes
   - Automatización completa
   - Monitoreo y debugging

### Próximos Pasos:

1. **Experimenta**:
   - Modifica hiperparámetros
   - Prueba otros modelos (GBT, SVM)
   - Agrega más features

2. **Escala**:
   - Conecta a cluster Spark real
   - Procesa datasets más grandes
   - Usa Spark Streaming

3. **Productiviza**:
   - Crea DAGs de producción
   - Implementa CI/CD
   - Agrega alertas y notificaciones

4. **Optimiza**:
   - Tuning de performance Spark
   - Optimización de queries
   - Gestión de recursos

### Recursos Adicionales:

- [Spark Documentation](https://spark.apache.org/docs/latest/)
- [MLflow Documentation](https://mlflow.org/docs/latest/index.html)
- [Airflow Documentation](https://airflow.apache.org/docs/)
- [Spark MLlib Guide](https://spark.apache.org/docs/latest/ml-guide.html)

---

**¡Felicidades! Has completado el tutorial de Spark + Airflow + MLflow** 🎉
